In [ ]:
import tensorflow as tf
import keras
from keras import Model, Sequential
from keras.layers import *
import matplotlib.pyplot as plt

# 1.Preparing the Data

In [ ]:
(x_train,y_train), (x_test,y_test) = keras.datasets.cifar100.load_data()

169001437/169001437 [==============================] - 13s 0us/step


In [ ]:
x_train.shape, x_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

### Preprocessing the Data

In [ ]:
x_train, x_test = x_train/255.0, x_test/255.0
y_train, y_test = keras.utils.to_categorical(y_train, 100), keras.utils.to_categorical(y_test, 100)

### Data Augmentation

In [ ]:
datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,          # Random rotation up to 15 degrees
    width_shift_range=0.1,      # Random horizontal shift
    height_shift_range=0.1,     # Random vertical shift
    horizontal_flip=True,       # Random horizontal flip
    zoom_range=0.1,             # Random zoom-in
)

datagen.fit(x_train)
ds_train = datagen.flow(x_train, y_train, batch_size=32)
len(ds_train)

1563

# 2.MODEL

### Single ResNet Block

In [ ]:
class BasicResNetBlock(Layer):
    def __init__(self, num_filters, strides=1):
        super(BasicResNetBlock, self).__init__()
        self.strides = strides

        # Define layers in the block
        self.conv1 = Conv2D(num_filters, kernel_size=(3, 3), strides=strides, padding='same')
        self.bn1 = BatchNormalization()
        self.relu1 = Activation('relu')

        self.conv2 = Conv2D(num_filters, kernel_size=(3, 3), strides=1, padding='same')
        self.bn2 = BatchNormalization()

        # 1x1 convolution for shortcut if strides > 1
        if strides > 1:
            self.shortcut_conv = Conv2D(num_filters, kernel_size=(1, 1), strides=strides, padding='same')
            self.shortcut_bn = BatchNormalization()

        self.add = Add()
        self.relu2 = Activation('relu')


    def call(self, inputs):

        # First convolutional block
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu1(x)

        # Second convolutional block
        x = self.conv2(x)
        x = self.bn2(x)

        # Apply shortcut
        if self.strides > 1:
            shortcut = self.shortcut_conv(inputs)
            shortcut = self.shortcut_bn(shortcut)
            x = self.add([x, shortcut])
        else:
            x = self.add([x, inputs])

        x = self.relu2(x)

        return x

### Complete ResNet Model

In [ ]:
class ResNet(Model):
    def __init__(self, num_classes):
        super(ResNet, self).__init__()
        self.conv1 = Conv2D(64, kernel_size=(7, 7), strides=2, padding='same')
        self.bn1 = BatchNormalization()

        self.stage1 = self.build_stage(64, 2)
        self.stage2 = self.build_stage(128, 2, strides=2)
        self.stage3 = self.build_stage(256, 2, strides=2)
        self.stage4 = self.build_stage(512, 2, strides=2)

        self.avgpool = GlobalAveragePooling2D()
        self.drop = Dropout(0.5)
        self.fc = Dense(num_classes, activation='softmax')


    # Builds a stage consist of given number of ResNet Blocks
    def build_stage(self, filters, num_blocks, strides=1):
        stage = Sequential(BasicResNetBlock(filters, strides=strides))
        if num_blocks>1:
            for _ in range(num_blocks-1):
                stage.add(BasicResNetBlock(filters))

        return stage


    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = ReLU()(x)

        # Residual Blocks
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)

        x = self.avgpool(x)
        x = self.drop(x)
        x = self.fc(x)
        return x

### Building the Model

In [ ]:
input_shape = (32, 32, 32, 3)   # Input shape for CIFAR-100
num_classes = 100               # Number of classes in CIFAR-100

model = ResNet(num_classes)
model.build(input_shape)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Training

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss for early stopping
    patience=5,           # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore the best model weights after stopping
)

In [ ]:
model.fit(ds_train, epochs=30, validation_data=(x_test, y_test), callbacks=[early_stopping])

Epoch 1/30
1563/1563 [==============================] - 90s 41ms/step - loss: 4.0889 - accuracy: 0.0821 - val_loss: 3.7551 - val_accuracy: 0.1251
Epoch 2/30
1563/1563 [==============================] - 60s 38ms/step - loss: 3.4424 - accuracy: 0.1703 - val_loss: 3.2337 - val_accuracy: 0.2170
Epoch 3/30
1563/1563 [==============================] - 61s 39ms/step - loss: 3.0963 - accuracy: 0.2350 - val_loss: 2.8562 - val_accuracy: 0.2758
Epoch 4/30
1563/1563 [==============================] - 61s 39ms/step - loss: 2.8126 - accuracy: 0.2879 - val_loss: 2.5956 - val_accuracy: 0.3358
Epoch 5/30
1563/1563 [==============================] - 61s 39ms/step - loss: 2.5799 - accuracy: 0.3355 - val_loss: 2.3289 - val_accuracy: 0.3882
Epoch 6/30
1563/1563 [==============================] - 61s 39ms/step - loss: 2.3940 - accuracy: 0.3729 - val_loss: 2.3751 - val_accuracy: 0.3890
Epoch 7/30
1563/1563 [==============================] - 60s 39ms/step - loss: 2.2535 - accuracy: 0.4059 - val_loss: 2.1639 -

### Saving the Model

In [ ]:
SAVE_PATH = '1.ResNet18'

tf.saved_model.save(model, SAVE_PATH)